In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install tensorflow==2.12.0

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Attention,Concatenate
from tensorflow.keras.models import Model
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.gleu_score import corpus_gleu
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-11-29 15:12:29.402821: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-29 15:12:29.433200: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-29 15:12:29.583918: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-29 15:12:29.584828: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-29 15:12:30.419794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
df=pd.read_csv("en_es_45k.csv")
df.head()

,English,Spanish
0,I declare resumed the session of the European ...,Declaro reanudado el período de sesiones del P...
1,"Although, as you will have seen, the dreaded '...","Como todos han podido comprobar, el gran ""efec..."
2,You have requested a debate on this subject in...,Sus Señorías han solicitado un debate sobre el...
3,"In the meantime, I should like to observe a mi...","A la espera de que se produzca, de acuerdo con..."
4,"Please rise, then, for this minute' s silence.",Invito a todos a que nos pongamos de pie para ...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149988 entries, 0 to 149987
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   English  149829 non-null  object
 1   Spanish  149559 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB


In [4]:
print(df["English"].apply(type).value_counts())

English
<class 'str'>      149829
<class 'float'>       159
Name: count, dtype: int64


In [5]:
df.English.isnull().sum(),df.Spanish.isnull().sum()

(159, 429)

In [6]:
df.dropna(inplace=True)

In [7]:
print(df["Spanish"].apply(type).value_counts())


Spanish
<class 'str'>    149401
Name: count, dtype: int64


In [8]:
np.random.seed(42)
tf.random.set_seed(42)

In [9]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

In [10]:
def tokenize_with_special_tokens(data, lang_key, num_words=None):
    tokenizer = Tokenizer(num_words=num_words, filters='', lower=True, oov_token="<unk>")
    special_tokens = ["<sos>", "<eos>"]
    tokenizer.fit_on_texts(special_tokens + data[lang_key].tolist())
    return tokenizer

In [11]:
en_tokenizer = tokenize_with_special_tokens(train_data, "English")
sp_tokenizer = tokenize_with_special_tokens(train_data, "Spanish")

In [12]:
def preprocess_with_special_tokens(data, en_tokenizer, sp_tokenizer, max_seq_len):
    en_sequences = en_tokenizer.texts_to_sequences(data['English'])
    sp_sequences = ["<sos> " + sent + " <eos>" for sent in data['Spanish']]
    sp_sequences = sp_tokenizer.texts_to_sequences(sp_sequences)

    en_sequences = pad_sequences(en_sequences, maxlen=max_seq_len, padding="post")
    sp_sequences = pad_sequences(sp_sequences, maxlen=max_seq_len + 2, padding="post")

    decoder_input = sp_sequences[:, :-1]
    decoder_target = sp_sequences[:, 1:]

    return en_sequences, decoder_input, decoder_target

In [13]:
max_seq_len = 20
en_train, dec_train_input, dec_train_target = preprocess_with_special_tokens(train_data, en_tokenizer, sp_tokenizer, max_seq_len)
en_val, dec_val_input, dec_val_target = preprocess_with_special_tokens(val_data, en_tokenizer, sp_tokenizer, max_seq_len)
en_test, _, _ = preprocess_with_special_tokens(test_data, en_tokenizer, sp_tokenizer, max_seq_len)

In [14]:
en_train.shape,dec_train_input.shape, dec_train_target.shape

((107568, 20), (107568, 21), (107568, 21))

In [15]:
embedding_dim = 128
hidden_units = 256
vocab_size_en = len(en_tokenizer.word_index) + 1
vocab_size_hi = len(sp_tokenizer.word_index) + 1

In [16]:
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=vocab_size_en, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding)

2024-11-29 15:12:51.875233: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-11-29 15:12:52.165866: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-29 15:12:52.167245: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMEN

In [17]:
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=vocab_size_hi, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[encoder_state_h, encoder_state_c])

2024-11-29 15:12:52.380271: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-29 15:12:52.382917: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-29 15:12:52.383936: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [18]:
attention_layer = Attention()
context_vector = attention_layer([decoder_outputs, encoder_outputs])

In [19]:
decoder_combined_context = Concatenate(axis=-1)([context_vector, decoder_outputs])
decoder_dense = Dense(vocab_size_hi, activation="softmax")
decoder_outputs = decoder_dense(decoder_combined_context)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 128)    7949440     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 128)    11752960    ['input_2[0][0]']                
                                                                                              

In [22]:
batch_size = 64
epochs = 100
history = model.fit(
    [en_train, dec_train_input], dec_train_target,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([en_val, dec_val_input], dec_val_target)
)

Epoch 1/10
1681/1681 [==============================] - ETA: 0s - loss: 4.9714 - accuracy: 0.3021

2024-11-29 15:36:49.556591: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-29 15:36:49.558039: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-29 15:36:49.559022: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1681/1681 [==============================] - 1334s 794ms/step - loss: 4.9714 - accuracy: 0.3021 - val_loss: 4.4974 - val_accuracy: 0.3335
Epoch 2/10
  87/1681 [>.............................] - ETA: 20:23 - loss: 4.2769 - accuracy: 0.3416

KeyboardInterrupt: 

In [ ]:
model.save("machineTranslation_eng_spa.h5")

In [ ]:
def predict_translation(input_text, model, en_tokenizer, sp_tokenizer, max_seq_len):
    input_seq = en_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_seq_len, padding="post")

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = sp_tokenizer.word_index['<sos>']

    decoded_sentence = []

    for _ in range(max_seq_len + 2):
        predictions = model.predict([input_seq, target_seq], verbose=0)
        predicted_token = np.argmax(predictions[0, -1, :])

        sampled_word = sp_tokenizer.index_word.get(predicted_token, '<unk>')
        if sampled_word == '<eos>':
            break

        decoded_sentence.append(sampled_word)

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = predicted_token

    return ' '.join(decoded_sentence)

In [ ]:
input_text = "who are you"
predicted_translation = predict_translation(input_text, model, en_tokenizer, sp_tokenizer, max_seq_len)
print(f"Input: {input_text}")
print(f"Predicted Translation: {predicted_translation}")

Input: who are you
Predicted Translation: de de de de de de de de de de de de de de de de de de de de de de


In [ ]:
def evaluate_bleu_score(model, data, en_tokenizer, sp_tokenizer, max_seq_len):
    references = []
    hypotheses = []

    for _, row in data.iterrows():
        input_text = row['English']
        reference = row['Spanish'].split()

        predicted_translation = predict_translation(input_text, model, en_tokenizer, sp_tokenizer, max_seq_len)
        hypothesis = predicted_translation.split()

        references.append([reference])
        hypotheses.append(hypothesis)

    bleu_score = corpus_bleu(references, hypotheses)
    corpus_gleu_score = corpus_gleu(references, hypotheses)
    return bleu_score,corpus_gleu_score

In [ ]:
test_test_data=test_data.head(100)

In [ ]:
bleu_score,corpus_gleu_score = evaluate_bleu_score(model, test_test_data, en_tokenizer, sp_tokenizer, 20)
print(f"Test BLEU Score: {bleu_score}"),f"Glue score :{corpus_gleu_score}"

Test BLEU Score: 8.700039049882008e-232


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

(None, 'Glue score :0.015388868411352745')